# Lesson 6: (UPDATE) Let's get inline and with Amazon Nova.

* This works with other models too, but we'll have a try with Amazon Nova.

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [ ]:
%autosave 0

# %%capture
%run ./ro_shared_data/lesson_2_prep.py lesson6

import os

region_name = 'us-west-2'

lambda_function_name = os.environ['LAMBDA_FUNCTION_NAME']
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

## Start of the lesson

In [ ]:
import boto3, uuid

In [ ]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

### Amazon Nova

In [ ]:
# UPDATE 1 - Meet Amazon Nova

# Amazon Nova is a new generation of state-of-the-art foundation models that deliver 
# frontier intelligence and industry leading price performance, available on Amazon Bedrock. 
# Amazon Nova models include three understanding and two creative content generation models.

# https://docs.aws.amazon.com/nova/latest/userguide/what-is-nova.html

foundation_model = 'us.amazon.nova-lite-v1:0'

### Amazon Bedrock Inline Agents

You can run this with any model that agents supports, we will take a look at using Amazon Nova.

In [ ]:
# UPDATE 2 - Inline Agents - Conduct rapid experimentation

# You can configure and invoke an inline Amazon Bedrock agent dynamically at runtime using 
# InvokeInlineAgent API. Using an inline agent provides you with flexibility to specify your 
# agent capabilities like foundation models, instructions, action groups, guardrails, and 
# knowledge bases at the same time you invoke your agent. You do not need to pre-define 
# agent capabilities before you can use it.

# https://docs.aws.amazon.com/bedrock/latest/userguide/agents-create-inline.html

action_groups = [
    {
        'actionGroupName': 'inline',
        'actionGroupExecutor': {
            'lambda': lambda_function_arn
        },
        'functionSchema': {
            'functions': [
                {
                    'name': 'customerId',
                    'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                    'parameters': {
                        'phone': {'description': 'Phone number', 'required': False, 'type': 'string'},
                        'name': {'description': 'Customer name', 'required': False, 'type': 'string'},
                        'email': {'description': 'Email address', 'required': False, 'type': 'string'}
                    },
                    'requireConfirmation': 'DISABLED'
                },
                {
                    'name': 'sendToSupport',
                    'description': 'Send a message to the support team, used for service escalation.',
                    'parameters': {
                        'supportSummary': {'description': 'Summary of the support request', 'required': True, 'type': 'string'},
                        'custId': {'description': 'customer ID', 'required': True, 'type': 'string'}
                    },
                    'requireConfirmation': 'DISABLED'
                }
            ]
        }
    }
]

instruction = "You are a front line customer support agent for our company. Your role is to process customer messages and route to a human customer support agent if action is required.\n\nWhen processing a customer message, follow these guidelines:\n1. Analyze the customer's message to understand their issue or query.\n2. Determine the appropriate action based on the nature and severity of the issue.\n3. Use the appropriate tools to process the request or route it to a human agent if necessary.\n4. Provide a clear, concise summary of your analysis and actions for internal review.\n\nIMPORTANT: Adhere to these strict rules in all interactions:\n- Never provide information or details that you are not 100% certain are factual and correct.\n- Do not make assumptions about company policies, procedures, or specific details unless they are explicitly provided to you.\n- If you are unsure about any aspect of a customer's query or the appropriate response, always route the conversation to a human agent.\n- When routing to a human agent, provide only the information given by the customer without adding any assumptions or speculations.\n- If asked about policies, procedures, or specific details that you don't have explicit information about, respond by stating that you'll need to route the query to a human agent for accurate information.\n- Your primary function is to route queries appropriately, not to provide detailed responses or solve issues directly unless you have been given explicit instructions and information to do so."

session_id = str(uuid.uuid4())

In [ ]:
# invoke_inline_agent
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_inline_agent.html

input_text = 'My name is Mike (mike@mike.com), my mug is broken and I want a refund.'

response = bedrock_agent_runtime.invoke_inline_agent(
    actionGroups=action_groups,
    enableTrace=True,
    endSession=False,
    foundationModel=foundation_model,
    instruction=instruction,
    inputText=input_text,
    sessionId=session_id
)

eventstream = response.get('completion')

for event in eventstream:
    if 'chunk' in event:
        print(event['chunk']['bytes'].decode('utf-8'))
    else: 
        print(".", end="")

In [ ]:
input_text = 'Thanks, I love your mugs, they are usually so great!'

response = bedrock_agent_runtime.invoke_inline_agent(
    actionGroups=action_groups,
    enableTrace=True,
    endSession=False,
    foundationModel=foundation_model,
    instruction=instruction,
    inputText=input_text,
    sessionId=session_id
)

eventstream = response.get('completion')

for event in eventstream:
    if 'chunk' in event:
        print(event['chunk']['bytes'].decode('utf-8'))
    else: 
        print(".", end="")

# Return Control - Local Tools

As well as using Lambda Functions, agents can use tools on your local machine, or somewhere accessible by the client code.  This is called 'return control'.

For more details on this you can get the docs [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-returncontrol.html?trk=64e03f01-b931-4384-846e-db0ba9fa89f5&sc_channel=el).

### What else?

With this method we can use it for more than just experimentation.  We could us it to dynamically change the agent's tool capability.

See: [Build a dynamic, role-based AI agent using Amazon Bedrock inline agents](https://aws.amazon.com/blogs/machine-learning/build-a-dynamic-role-based-ai-agent-using-amazon-bedrock-inline-agents/?trk=64e03f01-b931-4384-846e-db0ba9fa89f5&sc_channel=el)

Code: [Inline Agent HR Assistant](https://github.com/awslabs/amazon-bedrock-agent-samples/tree/main/examples/agents_ux/inline-agent-hr-assistant?trk=64e03f01-b931-4384-846e-db0ba9fa89f5&sc_channel=el)

### AWS Console Link

Want to see the AWS Console behind the lesson?  The permissions are limited... but you can get access here: 

In [ ]:
from IPython.display import HTML

console_url = os.environ['AWS_CONSOLE_URL']
params = "&trk=64e03f01-b931-4384-846e-db0ba9fa89f5&sc_channel=el"
full_url = console_url + params

HTML(f'<a href="{full_url}" target="_blank">Open AWS Console</a>')